# Labeling changepoints based on existing tags

In [29]:
import os.path
import copy
import pickle
import pandas
import numpy
import numpy.random

# Old-fashioned learning
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, f1_score

# Deep learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable

from intensix.monitor.models import Alert

In [30]:
DATA = "../data"
MODELS = "../models"

STAY_TAGS = os.path.join(DATA, "stay_tags.pkl")
SOMESTAYS = os.path.join(DATA, "stays-3-14-days")

with open(STAY_TAGS, "rb") as f:
    stay_tags = pickle.load(f)
    
with open(SOMESTAYS, "r") as f:
    somestays = []
    for line in f:
        somestays.append(line.strip())

## Function for labeling a single stay

In [31]:
def label_changepoints(stayid, df, changepoints):
    """Adds tag-based labels.
    """
    changepoints = numpy.append(changepoints, numpy.zeros_like(changepoints[:, -1:]), axis=1)
    if stayid in stay_tags:
        tags = sorted(
            set([(pandas.to_datetime(tag['time']) - df.index[0]).total_seconds()//60 
                 for tag in stay_tags[stayid]
                 if 'deterioration' in tag['concept']]))
        ia = 0
        it = 0
        while True:
            if it == len(tags):
                break
            if ia == len(changepoints):
                break
            if changepoints[ia, 0] > tags[it]:
                it += 1
                continue
            changepoints[ia, -1] = 1
            ia += 1
    return changepoints

## Loop over the dataset 

In [32]:
POS = 0
TOT = 0
dataset = []
for stayid in somestays:
    try:
        with open(os.path.join(DATA, "monitor-dataset-{}.pkl".format(stayid)),
                  "rb") as f:
            df = pickle.load(f)
        with open(os.path.join(DATA, "monitor-dataset-{}-changepoints.npy".format(stayid)),
                  "rb") as f:
            changepoints = numpy.load(f)
        print("+", end="")
        labeled_changepoints = label_changepoints(stayid, df, changepoints)
        numpy.save(os.path.join(DATA, "monitor-dataset-{}-labeled-changepoints.npy".format(stayid)),
                   labeled_changepoints)
        dataset.append(labeled_changepoints)
        POS += int(numpy.sum(labeled_changepoints[:, -1]))
        TOT += len(labeled_changepoints)
    except FileNotFoundError:
        print("-", end="")
print()
print("{} relevant changepoints out of {} total ({:.2f})".format(POS, TOT, POS/TOT))
dataset = numpy.concatenate(dataset, axis=0)
numpy.save(os.path.join(DATA, "labeled-changepoints.npy"), dataset)

-+++++++++++++++++++-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-++++++++++++++++++++++++++++++++++++++++++++++++++++++++-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-++++++++++++++++++++++++++++++++++++++++++++++++++++++-+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-+++++++++++++++++++++++++++++++++++++++-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-+-++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Baseline tests --- scikit-learn style classification

In [24]:
CV = 5
TF = 1 / CV
class_weight = {0: POS,
                1: TOT - POS}

### Logistic regression

In [25]:
model = LogisticRegression(class_weight=class_weight)
scores = cross_val_score(model, dataset[:, 2:-1], dataset[:, -1], cv=CV, scoring='f1')
print(scores.mean(), scores.std())

0.654100846818 0.0157867953263


In [26]:
ntrain = int((1 - TF) * len(dataset))
trainset = dataset[:ntrain]
testset = dataset[ntrain:]
model.fit(trainset[:, 2:-1], trainset[:, -1])
print("TRAIN:\n{}".format(confusion_matrix(trainset[:, -1], model.predict(trainset[:, 2:-1])) /
                          len(trainset)))
print("\nTEST:\n{}".format(confusion_matrix(testset[:, -1], model.predict(testset[:, 2:-1])) /
                           len(testset)))

TRAIN:
[[ 0.37052331  0.17085838]
 [ 0.15306002  0.30555829]]

TEST:
[[ 0.31646168  0.1711211 ]
 [ 0.16808578  0.34433144]]


### Linear SVM 

In [ ]:
model = SGDClassifier(class_weight=class_weight, tol=0.001)
scores = cross_val_score(model, dataset[:, 2:-1], dataset[:, -1], cv=CV, scoring='f1')
print(scores.mean(), scores.std())

In [ ]:
ntrain = int((1 - TF) * len(dataset))
trainset = dataset[:ntrain]
testset = dataset[ntrain:]
model.fit(trainset[:, 2:-1], trainset[:, -1])
print("TRAIN:\n{}".format(confusion_matrix(trainset[:, -1], model.predict(trainset[:, 2:-1])) / 
                          len(trainset)))
print("\nTEST:\n{}".format(confusion_matrix(testset[:, -1], model.predict(testset[:, 2:-1])) /
                          len(testset)))

## Deep learning

In [33]:
NEPOCHS = 100
BATCH_SIZE = 16
LEARNING_RATE = 0.0001

model = Alert(hidden_size=128, p=0.5)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

ntrain = int((1 - TF) * len(dataset))
trainset = torch.from_numpy(dataset[:ntrain])
testset = torch.from_numpy(dataset[ntrain:])

def truepred(model, dset):
    """Returns true and predicted labels.
    """
    y_true = dset[:, -1].numpy()
    y_pred = numpy.round(model(Variable(dset[:, 2:-1]))
                         .data.numpy()[:, 0])
    return y_true, y_pred
    
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
iepoch = 0

In [36]:
iepoch0 = iepoch
while iepoch != iepoch0 + NEPOCHS:
    model.train()
    train_loss = 0.
    train_samples = 0
    print("batch", end=" ")
    for ibatch, batch in enumerate(train_loader):
        optimizer.zero_grad()
        z = model(Variable(batch[:, 2:-1]))
        y = Variable(batch[:, -1]).resize(batch.size(0), 1)
        weight = (y * class_weight[1] + (1 - y) * class_weight[0])/(class_weight[0] + class_weight[1])
        loss = F.binary_cross_entropy(z, y, weight)
        loss.backward()
        train_loss += loss.data[0] * batch.size(0)
        train_samples += batch.size(0)
        optimizer.step()
        if (ibatch + 1) % (len(trainset) // BATCH_SIZE // 5) == 0:
            print("{}({}): {:.4f}".format(ibatch + 1, iepoch + 1, train_loss / train_samples), end=" ")
    print()
    train_loss /= train_samples
        
    model.eval()
    z = model(Variable(testset[:, 2:-1]))
    y = Variable(testset[:, -1]).resize(len(testset), 1)
    weight = (y * class_weight[1] + (1 - y) * class_weight[0])/(class_weight[0] + class_weight[1])
    loss = F.binary_cross_entropy(z, y, weight)
    print("EPOCH {}: train loss: {:.4f}, test loss: {:.4f}".format(iepoch + 1, train_loss, loss.data[0]))  
    iepoch += 1
    
    if iepoch % 10 == 0:
        train_true, train_pred = truepred(model, trainset)
        test_true, test_pred = truepred(model, testset)

        print("\nTRAIN:\n{}\nF1 = {:.4f}\n\nTEST:\n{}\nF1 = {:.4f}\n"
              .format(confusion_matrix(train_true, train_pred)/len(trainset),
                      f1_score(train_true, train_pred),
                       confusion_matrix(test_true, test_pred)/len(testset), 
                      f1_score(test_true, test_pred)))

batch 1672(34): 0.3044 3344(34): 0.3038 5016(34): 0.3038 6688(34): 0.3037 8360(34): 0.3040 
EPOCH 34: train loss: 0.3040, test loss: 0.3145
batch 1672(35): 0.3026 3344(35): 0.3023 5016(35): 0.3028 6688(35): 0.3034 8360(35): 0.3040 
EPOCH 35: train loss: 0.3040, test loss: 0.3147
batch 1672(36): 0.3032 3344(36): 0.3034 5016(36): 0.3034 6688(36): 0.3035 8360(36): 0.3037 
EPOCH 36: train loss: 0.3037, test loss: 0.3145
batch 1672(37): 0.3041 3344(37): 0.3043 5016(37): 0.3042 6688(37): 0.3039 8360(37): 0.3039 
EPOCH 37: train loss: 0.3039, test loss: 0.3143
batch 1672(38): 0.3044 3344(38): 0.3036 5016(38): 0.3041 6688(38): 0.3038 8360(38): 0.3037 
EPOCH 38: train loss: 0.3037, test loss: 0.3145
batch 1672(39): 0.3017 3344(39): 0.3035 5016(39): 0.3038 6688(39): 0.3035 8360(39): 0.3033 
EPOCH 39: train loss: 0.3033, test loss: 0.3144
batch 1672(40): 0.3034 3344(40): 0.3038 5016(40): 0.3040 6688(40): 0.3038 8360(40): 0.3037 
EPOCH 40: train loss: 0.3036, test loss: 0.3145

TRAIN:
[[ 0.3489367

KeyboardInterrupt: 

In [37]:
torch.save(model.state_dict(), os.path.join(MODELS, "changepoints.model"))